# Machine Learning with Collaborative Filtering

## Contents
1. **Loading** movie dataset
2. Creating your movie **profile**
3. **Predicting** ratings
4. Finding **recommendations**

In this challenge exercise, we will implement user-item collaborative filtering. 
We will first make a prediction for the rating you would give to a certain movie. Then we will generate recommendations, based on the movies that people similar to you liked.

You will need two data files, which can be found in the `moviedata` folder. The data is separated with tabs.
* `ratings.data`, which contains the ratings of 100,000 movies by many users (4 fields : user_id, movie_id, rating, timestamp).
* `movies.data`, a mapping between movie ID's and titles.

## 1. Loading movie dataset

First, we will load the `ratings.data` and `movies.data`. We will use the Python library [Pandas](http://pandas.pydata.org/) for this and load the data into a [DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

Find the function in Pandas that is used to read csv data and use it to load the ratings and titles. Don't forget to specify the separator!

(Hint: if you encounter encoding problems, set the encoding to 'cp1252'.)

In [ ]:
import pandas as pd

ratings =  # TODO: load moviedata/ratings.data
movies =  # TODO: load moviedata/movies.data

We can look at our new dataframes with the `.head()` function. This gives us the first five rows of the dataframe (unless you put a different number between the brackets).

In [ ]:
movies.head()

In [ ]:
ratings.head()

In our ratings dataframe, we don't really need the timestamp column. Remove this column from the dataframe.

In [ ]:
ratings =  # TODO: remove timestamp column

Now we will try to combine the ratings with the movie titles. We will write a function that can turn a column with movie id's into a column with movie names. By turning this into a function, we can easily apply it to other dataframes later.

In [ ]:
def add_titles(df, movie_titles_df):
    df =  # merge dataframes
    df =  # remove movie_id column
    return df

In [ ]:
ratings = add_titles(ratings, movies)
ratings.head()

You should not remove the movie id column without thinking though. This id could separate two movies with the same title (and year) that are not the same. However, in this case, this is not necessary so we can safely remove it.

## 2. Creating your own movie profile

Now we have a dataframe that describes how a lot of users rate movies. The idea of collaborative filtering is to find a users that are similar to you and look at the movies they like or don't like. Your own ratings can be predicted based on the ratings your "neighbours" give these movies.

So before we can find your neighbours, you need to rate a couple of movies. Let's create a movie profile for you.

### 2.1. Getting the most famous movies

The first step will be to filter our movies. To make sure you know the movies you're about to rate, we will find the 100 most reviewed movies.

Follow these steps:
- Group the ratings dataframe by title
- On these grouped items, apply the count function, this will give you a dataframe
- Select the rating column. (Hint: `df["colname"]`)
- Sort the values descending, so the most reviewed titles appear on top.
- Take the first 100 rows. (Hint: use `head()`)

You can do this all in one long chain and assign it to a new dataframe.

In [ ]:
most_famous_movies =  # TODO: group, count, select, sort and take first 100

Have a look at the result with `head()`. The most reviewed movie should have 583 reviews. Can you guess which one it is?

In [ ]:
# TODO: check result

You might also notice that this result is not displayed as a table anymore. That is because the result has only an index and one column, so pandas turned it into a [Series](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.html) instead. Luckily for us, series work almost exactly the same as dataframes.

### 2.2. Rating a few of the most famous movies

Now onto step 2: reviewing these movies. We will write a loop that will present a movie title to you and asks you to rate it. You don't want to rate all 100 movies though, let's just rate five of them.

First, we want a small function that allows you to enter a number. So write a function that:
- Gets input from the user. (Hint: experiment with `input("Hello")`.)
- Then check if the entered text is a number.
- If the text is a number between 1 and 5: return the number
- Else: return None (python's null value)

In [ ]:
def get_int_input(descr):
    # TODO: write function

Now we write a loop that keeps showing you movie titles until you have rated 5 of them.

Here are the steps:
- Create a new, empty dataframe with columns "title" and "rating".
- Shuffle the most famous movies. (Hint: check pandas' `sample` function)
- Ask rating for every movie.
- Append result to the new dataframe if it is not None. (Hint: it's easy to append a dict to a df if you set `ignore_index` to True)
- If we have enough ratings: break the loop.

When you are done you can run the cell and rate five movies. You can skip the ones you don't know by pressing enter, or anything other than a number.

In [ ]:
user_ratings =  # TODO: create new dataframe
ratings_needed = 5
shuffled_movies =  # TODO: shuffle most_famous_movies

for title, row in shuffled_movies.iteritems():
    rating = get_int_input("What rating (1-5) would you give to the movie " + title + "? ")
    
    # TODO: append rating to user_ratings if it is not None
    
    if len(user_ratings) >= ratings_needed:
        break

Now you can view your profile:

In [ ]:
user_ratings

## 3. Predicting ratings

Based on these ratings, we will predict your opinion about other movies. The general approach to find your rating for a certain movie, is to look at all the users that have rated this movie and then take the average of their ratings for this movie. We multiply their rating by a weight though, based on how similar the users's ratings are to your own.

Now let's do that step by step.

We begin with the most basic function: getting a movie title based on an id. This will make it easier for us to test our code, because we won't need to type the entire movie title every time.

Write a function that uses the `movies` dataframe to return the title of a movie id. You'll need some pandas for this.

In [ ]:
def get_movie_title(movie_id):
    # TODO: complete function

Now we can use the function to select a movie:

In [ ]:
selected_movie = get_movie_title(1)
selected_movie

We will have a look at your predicted rating for this movie. If you rated this one, you can change the movie id.

### 3.1. Normalising ratings

The next important part is to normalise your ratings. Basically you divide them all by the average. We do this because, if you always gave movies five stars, that says less about your taste than if you rated some higher and some lower. It also centers your ratings around zero. A negative rating means you enjoyed it less than usual, a positive rating means you liked it more than usual.

From your user_ratings dataframe, calculate the average of your rating column. Store this as a variable, we will need it later.

In [ ]:
user_avg_rating =  # TODO: calculate your rating average
print(user_avg_rating)

### 3.2. Finding voters

Now we will have a look at other people who rated our movie. We will refer to them as voters. 

Filter the ratings dataframe by our selected movie title and then get the id's of those users.

In [ ]:
voters =  # TODO: find voters id's in rating df

In [ ]:
print("Number of voters who rated {}: {}".format(selected_movie, len(voters)))

If you didn't change the selected movie, there should be 452 voters.

### 3.3. Comparing one voter to you

We will focus on one voter first and compare him to you. After that, we will generalise it for all voters. We will select the first voter for now. We will assign his `user_id` to a variable so we can use it later.

In [ ]:
voter = voters.iloc[0]  # iloc[0] gets the element at index 0
voter

Now get all the voter's ratings from the ratings dataframe.

In [ ]:
voter_ratings =  # TODO: get voter's ratings
voter_ratings.head()

We would like to know how similar the voter is to you. If you have rated no movies in common, there is no point in considering him. So let's compare your ratings to his.

Combine your ratings with pandas' `merge` function (or alternatives).

In [ ]:
mutual_ratings =  # TODO: merge voter_ratings and user_ratings
mutual_ratings

Do you and the voter have the same taste, or in other words: do you have many movies in common? We would like to put a number to that question. So let's calculate the correlation coëfficient of your ratings. This says how similar two arrays are and is a function that can be found in numpy. Google a bit until you find the right function. This function gives you a matrix of multiple values as result, you need the element at `[0,1]`.

Hint: Get your ratings from the mutual_ratings dataframe and convert them to a list.

In [ ]:
import numpy as np

corr =  # TODO: calculate correlation between your ratings and voter's ratings
corr

If this number is NaN (not a number), then you have no movies in common with this voter or there is not enough distinction between your or his ratings. Just go back, pick another voter and run all the cells up to this point again. We will deal with these cases later.

Last but not least, we need the voter's rating for our movie and his average rating. Calculate those two and store them in variables.

In [ ]:
voter_movie_rating =  # TODO: find voter's rating for selected_movie
print(voter_movie_rating)

voter_avg_rating =  # TODO: calculate voter's average rating
print(voter_avg_rating)

We're almost done with this step. Now comes the magic math: we multiply the voter's normalised rating with his correlation with you. This we divide by the absolute correlation. Then we add your average rating to un-normalise it for you and there is your rating, based on one single voter.

In [ ]:
correlated_rating = corr * (voter_movie_rating - voter_avg_rating)
user_movie_rating = user_avg_rating + (correlated_rating / abs(corr))
user_movie_rating

### 3.4. Comparing all voters to you

We don't want to base our rating on one single voter though. We will edit the code a bit to base it on all users that have rated the movie.

We will:
- Loop over all the voters.
- Calculate the same things we did before for one voter.
- Ignore voters with a correlation of NaN.
- Sum the absolute correlation and the correlated rating for all voters and then divide them in the end.
- Turn this all into a function that we can use again later

In [ ]:
def predicted_rating(user_ratings, ratings, selected_movie):
    abs_corr_sum = 0
    corr_rating_sum = 0
    
    voters =  # TODO: find voters id's in rating df
    
    for voter in voters:
        # get voter's movies
        voter_ratings =  # TODO: get voter's ratings

        # check similarity to user not nan
        mutual_ratings =  # TODO: merge voter_ratings and user_ratings
        corr =  # TODO: calculate correlation between your ratings and voter's ratings

        if not np.isnan(corr):
            # get selected movie rating
            voter_movie_rating =  # TODO: find voter's rating for selected_movie

            # get voter's average
            voter_avg_rating =  # TODO: calculate voter's average rating

            # sum similarity
            abs_corr_sum += abs(corr)
            corr_rating_sum += corr * (voter_movie_rating - voter_avg_rating)

    # calculate predicted rating
    user_movie_rating = user_avg_rating + (corr_rating_sum / abs_corr_sum)
    return user_movie_rating

Now we can use our function to get the actual prediction for the movie:

In [ ]:
print("Predicted rating for {}: {}".format(selected_movie, round(predicted_rating(user_ratings, ratings, selected_movie))))

In [ ]:
# If you get a lot of warnings (these come from the correlation function), you can run this cell to turn all warnings off.
import warnings
warnings.filterwarnings('ignore')

You are now ready to make predictions! Try it for different movies, and see if it accurately reflects your tastes. If it does, you can use it to decide which movie to watch tonight! (Bear in mind though that this rating is only based on five ratings of yours, so it might not always be accurate.)

In [ ]:
movie = get_movie_title(2)
print("Predicted rating for {}: {}".format(movie, round(predicted_rating(user_ratings, ratings, movie))))

## 4. Finding recommendations

Now we can predict ratings for a specific movie, but when you're wondering what to watch you won't know which movie to look for. We could calculate the predicted rating for every single movie, but there is another solution as well. We could find the voters that are most similar to you, your neighbours, and see which movies they liked most.

### 4.1. Finding your neighbours

Calculate your correlation with every user in the ratings dataframe.

In [ ]:
corr_list = []
for voter in ratings.user_id.unique():
    voter_ratings =  # TODO: get voter's ratings
    mutual_ratings =  # TODO: merge voter_ratings and user_ratings
    corr =  # TODO: calculate correlation between your ratings and voter's ratings
    corr_list.append([voter, len(mutual_ratings), corr])
voter_corr = pd.DataFrame(corr_list, columns=["user_id", "movies_in_common", "corr"])

Now we have a dataframe of our similarity with all users.

In [ ]:
voter_corr.head()

We want to find out which users are most similar to us. In this case, we want the ten users with the most movies in common and the highest correlation.

Create a list of similar voters:
- Sort the `voter_corr` dataframe, by `movies_in_common` and then by `corr`. Sort the values descending so the highest ones appear on top. 
- Take the first ten rows to get the most similar users.
- Select the `user_id` column. 
- Cast this result to a list.

In [ ]:
similar_voters =  # TODO: create list of most similar user_id's
print(similar_voters)

Those are the id's of your neighbours.

### 4.2. Finding your neighbours favourite movies

Now we have a look at those users and the movies they gave high ratings.

Follow these steps:
- Start from the ratings dataframe.
- Filter out user_id's that don't appear in `similar_voters`.
- Filter out titles that appear in `user_ratings.title`.
- Group by title.
- Aggregate by both count and mean.
- Select rating column.
- Filter out movies with a mean rating of less than 4.
- Sort the result, on count and then on mean, descending.
- Take the top 10 rows.

In [ ]:
recommendations =  # TODO: filter voters and already seen movies from ratings df
recommendations =  # TODO: group by title and aggregate
recommendations =  # TODO: filter mean ratings less than 4
recommendations =  # TODO: sort and take first 10
recommendations

There you go, you have just built your own recomendation engine from scratch. Time to start making money!